##  Self-Supervised Consistency Loss

* Exploring Simple Siamese Representation Learning / Algorithm 1 SimSiam Pseudocode, PyTorch-like


##  Self-Supervised Consistency Loss

* Exploring Simple Siamese Representation Learning / Fig. 1
* 输入：two randomly augmented views $x_1$ and $x_2$ from an image $x$
* encoder $f$
    1. a backbone
    1. a projection MLP head
* predictor $h$ / MLP



### Siamese

* 连体双胞胎的

* $p_1\triangleq h(f(x_1))$ and $z_2\triangleq f(x_2)$
* negative cosine similarity
$$
\mathcal{D}(p_1,z_2)=-\frac{p_1}{\|p_1\|}\cdot\frac{z_2}{\|z_2\|}
$$


### Siamese

* symmetrized loss
$$
\mathcal{L}=\frac{1}{2}\mathcal{D}(p_1,\mathrm{stopgradient}(z_2))+\frac{1}{2}\mathcal{D}(p_2, \mathrm{stopgradient}(z_1))
$$

### Hypothesis

* 
$$
\mathcal{L}(\theta,\eta)=\mathop{\mathbb{E}}_{x,\mathcal{T}}\bigl[\|\mathcal{F}_\theta(\mathcal{T}(x))-\eta_x\|^2\bigr]
$$
* $\mathcal{F}$ is a network parameterized by $\theta$
* $\mathcal{T}$ is the augmentation
* $x$ is an image
* $\eta_x$ is the representation of the image $x$ (intuitively)


### Hypothesis

* 
$$
\begin{aligned}
\theta^t&\gets \mathop{\arg\,\min}_\theta\mathcal{L}(\theta,\eta^{t-1})\\
\eta^t&\gets \mathop{\arg\,\min}_\eta\mathcal{L}(\theta^t,\eta)
\end{aligned}
$$


### Hypothesis

* minimize $\mathop{\mathbb{E}}_\mathcal{T}\bigl[\|\mathcal{F}_{\theta^t}(\mathcal{T}(x))-\eta_x\|^2\bigr]$
* easy to solve by 
$$
\eta^t_x\gets \mathop{\mathbb{E}}_\mathcal{T}\bigl[\mathcal{F}_{\theta^t}(\mathcal{T}(x))\bigr]
$$

$\eta_x$ is assigned with the average representation of $x$ over the distribution of augmentation

### Hypothesis

* ignoring $\mathop{\mathbb{E}}_\mathcal{T}[\cdot]$
$$
\eta^t_x\gets \mathcal{F}_{\theta^t}(\mathcal{T'}(x))
$$
* One-step alternation
$$
\theta^{t+1}\gets \mathop{\arg\,\min}_\theta\mathop{\mathbb{E}}_{x,\mathcal{T}}\bigl[\|\mathcal{F}_\theta(\mathcal{T}(x))-\mathcal{F}_{\theta^t}(\mathcal{T'}(x))\|^2\bigr]
$$
* the expectation $\mathop{\mathbb{E}}_\mathcal{T}[\cdot]$ is ignored / The usage of $h$ may fill this gap. / learn to predict the expectation

##  Self-Supervised Consistency Loss

* taking **one-step** rollout as an example
* $$\mathcal{L}_\mathrm{similarity}(s_{t+1},\hat{s}_{t+1})=\mathcal{L}_2(sg(P_1(s_{t+1})),P_2(P_1(\hat{s}_{t+1})))$$
* $\mathcal{L}_2$ is the negtive cosine similarity loss
* $sg(P_1)$ means stopping gradients

##  Self-Supervised Consistency Loss

* dynamics network
    * MuZero中为：$r^k,s^k=g_\theta(s^{k-1},a^k)$
    * EfficientZero中为：$\hat{s}_{t+1}=\mathcal{G}(s_t,a_t)$
* representation network
    * MuZero中为：$s^0=h_\theta(o_1,\dotsc,o_t)$
    * EfficientZero中为：$s_t=\mathcal{H}(o_t),s_{t+1}=\mathcal{H}(o_{t+1})$

## End-To-End Prediction of the Value Prefix

* MuZero
    * Monte-Carlo tree search (Appendix B Search)
    * Backup中：
    $$
    G^k=\sum_{\tau=0}^{l-1-k}\gamma^\tau r_{k+1+\tau}+\gamma^{l-k}v^l
    $$
    * $r^k,s^k=g_\theta(s^{k-1},a^k)$ / the compounding error in the recurrent rollouts


* EfficientZero
    * $$Q(s_t,a)=\sum_{i=0}^{k-1}\gamma^ir_{t+i}+\gamma^kv_{t+k}$$
    * $r_{t+i}$ is the reward **predicted** from unrolled state $\hat{s}_{t+i}$
    * value prefix $\sum_{i=0}^{k-1}\gamma^ir_{t+i}$
    * *value prefix* prediction network: $r_t,h_{t+1}=\mathcal{R}(\hat{s}_{t+1},h_t)$
        * why $h_t$? LSTM.
        * how to train? $\mathcal{L}(u_t,r_t)$, where $\mathcal{L}$ is the total loss of the unrolled 5 steps
        * $u_t$ is the reward from the environment

## End-To-End Prediction of the Value Prefix

* Mastering Atari Games with Limited Data / Fig. 3
* If we only see **the first observation**, along with **future actions**, it is **very hard** both for an agent and a human to predict at which **exact** future timestep the player would lose a point.
* value prefix $\sum_{i=0}^{k-1}\gamma^ir_{t+i}$ is -1
* which $r_{t+i}$ is -1

* However, it is easy to predict the agent will miss the ball after a sufficient number of timesteps if he does not move.